# library

In [ ]:
from langchain_ollama import ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
# import os
# from dotenv import load_dotenv
# load_dotenv()

True

In [3]:
# os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

llm = ChatOllama(model="qwen2.5:7b")

# repo_id = "mistralai/Mistral-7B-Instruct-v0.3"
# llm = HuggingFaceEndpoint(repo_id=repo_id, max_length=150, temperature=0.7, token=os.getenv("HF_TOKEN"))

# Use HuggingFace's MiniLM embeddings (384 dimensions)
# embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

d:\vedyon\langchain_env\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


d:\vedyon\langchain_env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
parser = StrOutputParser()

review_compression_prompt = """ 
Give sentiment and themes for the given input reviews.
Give output as json with 'sentiment' key and value should be either of [POSITIVE, NEGATIVE, NEUTRAL].
and another key 'theme' with list of themes talked about in the reviews, for example 'product issue', 'packing issues' etc...

Strictly only give JSON output as a string and nothing else. Do not use markdown.

Input reviews:
``` {input_review} ```
"""

In [10]:
input_review = "this is the bedsheet is beatuful and looks just like ain the picture. The print is very pretty, it is really comfortable ad smooth"

promt_temp = PromptTemplate(template = review_compression_prompt, input_variables=["input_review"])

# input_prompt = promt_temp.invoke({'input_review': input_review})
# print(input_prompt.text)

formatted_prompt = promt_temp.format(input_review=input_review)
print(formatted_prompt)

 
Give sentiment and themes for the given input reviews.
Give output as json with 'sentiment' key and value should be either of [POSITIVE, NEGATIVE, NEUTRAL].
and another key 'theme' with list of theses talked about in the rviews, for example 'product issue', 'packing issues' etc...

strickly only give json output as strings and another nothing else. Do not give markdown.

input reviews:
``` this is the bedsheet is beatuful and looks just like ain the picture. The print is very pretty, it is really comfortable ad smooth ```



In [ ]:
# chain = promt_temp | llm | parser
# result = chain.invoke({ input_review:"input_review"})
# print(result)

In [12]:
result = llm.invoke(formatted_prompt)
compressed_review = parser.invoke(result)

print(compressed_review)

{"sentiment":"POSITIVE","theme":["appearance","comfort"]}


# update


In [13]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from enum import Enum
from typing import List

### with pydantic

In [25]:
# class Sentiment(str, Enum):
#     positive = "POSITIVE",
#     negative = "NEGATIVE",
#     neutral = "NEUTRAL",

# class compresed_review(BaseModel):
#     sentiment : Sentiment = Field(description="Gives sentiment value of the review")
#     theme: list = Field(description="theme expressed in the reviews")

# Define sentiment Enum properly
class Sentiment(str, Enum):
    positive = "POSITIVE"
    negative = "NEGATIVE"
    neutral = "NEUTRAL"

# Define the compressed review model
class CompressedReview(BaseModel):
    sentiment: Sentiment = Field(description="Gives sentiment value of the review")
    theme: list = Field(description="Themes expressed in the reviews")

In [26]:
parser_json = JsonOutputParser(pydantic_object=CompressedReview)

In [27]:
# prompt = PromptTemplate(
#     template = "Gives sentiment and theme for a reviews. \n {format_instructions} \n {input_review}",
#     input_variable = ['input_review'],
#     partial_variables={"format_instructions": parser_json.get_format_instructions()},
# )

# Define the prompt correctly
prompt = PromptTemplate(
    template="Gives sentiment and theme for a review. \n{format_instructions} \n{input_review}",
    input_variables=["input_review"],
    partial_variables={"format_instructions": parser_json.get_format_instructions()},
)

In [28]:
print(parser_json.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Sentiment": {"enum": ["POSITIVE", "NEGATIVE", "NEUTRAL"], "title": "Sentiment", "type": "string"}}, "properties": {"sentiment": {"$ref": "#/$defs/Sentiment", "description": "Gives sentiment value of the review"}, "theme": {"description": "Themes expressed in the reviews", "items": {}, "title": "Theme", "type": "array"}}, "required": ["sentiment", "theme"]}
```


In [33]:
input_review = "this is the bedsheet is beatuful and looks just like ain the picture. The print is very pretty, it is really comfortable ad smooth"

formatted_prompt = prompt.invoke({'input_review': input_review})
print(formatted_prompt.text)

Gives sentiment and theme for a review. 
Return a JSON object. 
this is the bedsheet is beatuful and looks just like ain the picture. The print is very pretty, it is really comfortable ad smooth


In [34]:
result = llm.invoke(formatted_prompt)
compresed_review = parser_json.invoke(result)

In [36]:
compresed_review

{'sentiment': 'Positive', 'theme': ['Appearance', 'Comfort', 'Quality']}

### without pydantic

In [29]:
parser_json = JsonOutputParser()

# Define the prompt correctly
prompt = PromptTemplate(
    template="Gives sentiment and theme for a review. \n{format_instructions} \n{input_review}",
    input_variables=["input_review"],
    partial_variables={"format_instructions": parser_json.get_format_instructions()},
)

In [31]:
chain = prompt | llm | parser_json
review = "Why don't scientists trust atoms? Because they make up everything"
chain.invoke({'input_review': review})

{'sentiment': 'Neutral', 'theme': 'Joke about scientists and atoms'}